In [28]:
import re

import pandas as pd
import sys
import keras
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import numpy as np
from sqlalchemy import TextClause

sys.path.insert(1, r"..\models")
sys.path.insert(2, r"..\etl")

from connect_db import connect_to_grad_db

In [4]:
model = keras.models.load_model('../../saved_models/2dcnn_32_16_32_250k_rows/')

In [6]:
engine = connect_to_grad_db()

In [8]:
with engine.connect() as conn:
    query = TextClause("select content from news_api")
    content = conn.execute(query).fetchall()

In [10]:
def ready_text_for_pred(text):
    processed_text = re.sub(r" +", " ", (re.sub(r"[^a-zA-Z]", " ", text)).lower().strip())
    return processed_text

def preprocess_article_content(content):
    content = content.strip("\n")

    # Remove all special characters except sentence defining punctuation
    cleaned_content = re.sub(r"[^.!?\w]", " ", content)

    # Remove multiple spaces
    cleaned_content = re.sub(r"[ ]+", " ", cleaned_content)

    # Split on sentence punctuation
    split_sentences = re.split(r"[.!?]", cleaned_content)

    
    return split_sentences

In [49]:
def create_preds(article_content):
    sentences = preprocess_article_content(article_content)
    cleaned_sentences = [ready_text_for_pred(x) for x in sentences]
    preds = [model.predict([x]) for x in cleaned_sentences]

    sentences_w_preds = []
    for sentence,pred in zip(sentences, preds):
        if pred > 4:
            pred = 4
        elif pred < 0:
            pred = 0
        sentences_w_preds.append(sentence + " " + str(pred[0]))
    
    mean_pred = np.mean(preds)
    
    return ". ".join(sentences_w_preds), mean_pred

In [50]:
final_text, mean_pred = create_preds(content[0]._data[0])

1/1 [==============================] - 0s 63ms/step


In [60]:
df = pd.read_sql_table(
    "news_api", 
    engine.connect(), 
    columns=["content", "news_api_id"]
)

In [68]:
preds = df["content"].apply(create_preds)

1/1 [==============================] - 0s 61ms/step


KeyboardInterrupt: 

2.160657